In [ ]:
import pandas as pd


: 

In [1]:
import pandas as pd
import pyspark as ps
sc = ps.SparkContext()
from pyspark.sql import SparkSession


# Create a SparkSession
spark = SparkSession.builder.appName("MyApp").getOrCreate()


22/12/18 21:59:38 WARN Utils: Your hostname, Nikolass-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.3.24.19 instead (on interface en0)
22/12/18 21:59:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/18 21:59:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/18 21:59:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
locationIdToName = "./taxi+_zone_lookup.csv"
taxis = "./taxis"


In [3]:
# create a rdd file from the csv file using pyspark
dff_locactions = spark.read.csv(locationIdToName, header=True, inferSchema=True)
dff_locactions.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [5]:
# create a rdd file from the parquet file using pyspark
dff_taxis = spark.read.parquet(taxis)
dff_taxis.show()



+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2022-03-01 02:13:08|  2022-03-01 02:24:35|            1.0|          2.4|       1.0|                 N|          90|         209|           2|       10.0|  3.0|    0.5|       0.

In [6]:
# create a view for taxis 
dff_taxis.createOrReplaceTempView("taxis")
# do the same for locations
dff_locactions.createOrReplaceTempView("locations")

In [7]:
import time
start = time.time()
# lets filter all taxis trips , that happend in march
marchTrips = spark.sql(
  """
  SELECT *
  FROM taxis 
  INNER JOIN locations ON taxis.DOLocationID = locations.LocationID
  WHERE tpep_pickup_datetime >= '2022-03-01' 
  AND tpep_pickup_datetime < '2022-04-01' and tpep_dropoff_datetime >= '2022-03-01' 
  AND tpep_dropoff_datetime < '2022-04-01'
  AND locations.Zone = "Battery Park"
  ORDER BY Tip_amount DESC
  limit 1
  """
  )

bestTipTripMarch = ((marchTrips).collect())
end = time.time()
print(bestTipTripMarch)
print('Time: ',end - start)

# filter the previus trips that 

[Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2022, 3, 17, 12, 27, 47), tpep_dropoff_datetime=datetime.datetime(2022, 3, 17, 12, 27, 58), passenger_count=1.0, trip_distance=0.0, RatecodeID=1.0, store_and_fwd_flag='N', PULocationID=12, DOLocationID=12, payment_type=1, fare_amount=2.5, extra=0.0, mta_tax=0.5, tip_amount=40.0, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=45.8, congestion_surcharge=2.5, airport_fee=0.0, LocationID=12, Borough='Manhattan', Zone='Battery Park', service_zone='Yellow Zone')]
Time:  2.67386794090271


In [8]:
start = time.time()
highestTollFees = spark.sql(
  """
  SELECT MAX(Tolls_amount) as max_toll_fees_per_month, MONTH(tpep_pickup_datetime) as month
  FROM taxis
  where Tolls_amount > 0
  GROUP BY MONTH(tpep_pickup_datetime)
  """
  )
highestTollFees = ((highestTollFees).collect())
end = time.time()
print(highestTollFees)
print('Time: ',end - start)

[Row(max_toll_fees_per_month=193.3, month=1), Row(max_toll_fees_per_month=235.7, month=3), Row(max_toll_fees_per_month=911.87, month=4), Row(max_toll_fees_per_month=95.0, month=2), Row(max_toll_fees_per_month=800.09, month=6), Row(max_toll_fees_per_month=29.55, month=7), Row(max_toll_fees_per_month=18.3, month=10), Row(max_toll_fees_per_month=813.75, month=5)]
Time:  1.1666901111602783
